# Before We Start
1. Python, Pandas, VS Code setup instructions 
2. Dataset link or simulated pose data provided 

# Step 0: Install & Import Modules

In [ ]:
# Install stuff

In [2]:
# Import stuff
import pandas as pd
from pandas import DataFrame
import copy, json
from os import listdir

In [3]:
# Helper indexer that will convert from a joint number to the joint name, using COCO Pose Output Format
# This may or may not be helpful later for debugging
coord_to_name = ["head",
                 "hips",
                 "left_shoulder",
                 "left_elbow",
                 "left_hand",
                 "right_shoulder",
                 "right_elbow",
                 "right_hand",
                 "left_hip",
                 "left_knee",
                 "left_foot",
                 "right_hip",
                 "right_knee",
                 "right_foot",
                 "left_eye",
                 "right_eye",
                 "left_ear",
                 "right_ear"]

# Step 1: Load and examine pose data 
_Reviewed with Dr. Jin Chang. Recommendation: Add descriptions to tie back to presentation for every step and research Event Detection for real world application_

In [4]:
# Index downloaded pose data from OpenPoses.com
def index_poses(path="poses"):
    json_list = listdir(path)
    json_list.sort()        # Needed to prevent the JSONs from being in a seemingly random order
    return json_list

json_list = index_poses()

# Extract pose keypoints from all of the JSONs
def read_all_poses(json_list,path="poses"):
    pose_list = []
    for i in range(len(json_list)):
        item = path + "/" + json_list[i]
        with open(item) as file:
            data = json.load(file)
            points_list = list(data[0]["people"][0]["pose_keypoints_2d"])   # Hardcoded to fit the JSON scheme
            unneeded_3d_coord = 1.0   # the JSONs provide z-axis data, which we don't need, and it's all 1.0
            points_list = [i for i in points_list if i != unneeded_3d_coord]
            pose_list.append(points_list)
    return pose_list

pose_list = read_all_poses(json_list)

# Generate labels to use in the DataFrame
def generate_column_labels(coords=18):
    labels = []
    for i in range(coords):
        formatted_number = f"{i:02d}"           # Formats to 2 digit string representation of int, which will only work well to 99 points
        labels.append(formatted_number + "x")
        labels.append(formatted_number + "y")
    return labels

labels = generate_column_labels()

# Create DataFrame with proper coordinate labels
def prep_dataframe(labels):
    return DataFrame(columns=labels)

df = prep_dataframe(labels)

# Store each pose in the DataFrame
def store_pose(pose, labels, df):
    if len(pose) != len(labels):
        print("Error: Size mismatch between coordinates list and labels")
        return df
    temp_points = copy.copy(pose)   # Use a copy so that we don't potentially lose poses if we reuse the variable
    df.loc[len(df)] = temp_points   # Append to df. No need to return; it's an in-place edit

def store_all_poses(pose_list, labels, df):
    for pose in pose_list:
        store_pose(pose, labels, df)

store_all_poses(pose_list, labels, df)
print(df)


           00x         00y         01x         01y         02x         02y  \
0   416.970074  111.688862  395.498931  200.326142  343.197430  201.977769   
1   370.682954  112.752594  397.372456  176.547014  332.927073  176.547014   
2   298.092040  133.001079  306.938936  193.033584  253.857562  192.401663   
3   401.390829  113.188565  410.100915  170.675131  361.905107  172.417148   
4   371.250272  179.687440  349.656885  220.539793  288.378355  225.208634   
5   394.256385  157.521886  407.238060  220.807551  360.720392  210.530391   
6   343.242699  110.979140  382.994881  178.207095  321.028244  183.468413   
7   377.927925  105.643599  383.056230  183.707791  325.505256  189.975719   
8   380.413173  116.665346  384.598824  198.808747  324.953298  198.285540   
9   396.912812  110.173967  395.756666  207.868248  317.138783  206.134030   
10  405.833455  110.839554  388.264989  194.714816  337.826487  185.647220   
11  419.768267   98.331520  400.757826  180.134025  356.400129  

# Step 2: Clean and filter incomplete rows 

In [5]:
# Figure out which rows are incomplete so we know what we're about to get rid of
def print_empties(df):
    df_nan = df[df.isna().any(axis=1)] 
    if df_nan.empty:
        print("No incomplete rows")
    else:
        print(df_nan)

print_empties(df)

# Now, get rid of them
df.dropna(inplace=True)

No incomplete rows


# Step 3: Create derived features: speed, angle, joint distance 

In [23]:
# Calculate movement vectors for each change and export as new DataFrame
def coords_to_vectors(df):
    labels = df.columns
    vects = DataFrame(columns=labels)
    for i in range(0,len(df)-1):            # Outer loop to handle the rows
        pose1 = df.iloc[i].values.tolist()
        pose2 = df.iloc[i+1].values.tolist()
        movement = []
        for j in range(len(pose1)):
            movement.append(pose2[j] - pose1[j])
        vects.loc[len(df)] = movement
        vects.reset_index(drop=True, inplace=True)
    return vects

vectors = coords_to_vectors(df)
display(vectors)

,00x,00y,01x,01y,02x,02y,03x,03y,04x,04y,...,13x,13y,14x,14y,15x,15y,16x,16y,17x,17y
0,-46.287120,1.063732,1.873525,-23.779129,-10.270358,-25.430755,-26.933201,-45.533782,5.870077,-183.011075,...,164.055474,40.689105,-44.992294,-2.946021,-31.266525,-8.354571,-18.871088,-15.556504,16.897671,-25.473363
1,-72.590914,20.248485,-90.433520,16.486570,-79.069510,15.854649,-97.812728,-74.868351,40.396044,-26.894524,...,205.887399,-234.263467,-70.946224,18.063563,-69.142150,25.837040,-75.735000,20.476993,-84.034617,34.798189
2,103.298789,-19.812514,103.161979,-22.358454,108.047545,-19.984515,132.267300,72.890072,-65.811926,114.186876,...,-98.133375,230.348192,100.737232,-17.523697,99.746130,-24.594263,105.348738,-20.358874,110.983461,-34.295011
3,-30.140557,66.498875,-60.444030,49.864662,-73.526752,52.791486,-66.969257,-24.062556,23.474867,-106.911011,...,-170.622216,-11.141147,-30.210941,64.105801,-33.583534,69.912525,-54.769276,65.290607,-57.942447,79.226745
4,23.006113,-22.165554,57.581175,0.267758,72.342037,-14.678242,75.074888,-25.752588,152.851017,-15.703305,...,138.161554,18.346050,27.276378,-16.813483,26.735511,-21.140708,54.634634,-14.991486,51.190035,-19.318711
5,-51.013687,-46.542745,-24.243180,-42.600456,-39.692148,-27.061978,-49.578111,33.048994,-84.141834,21.698423,...,-31.699416,3.810633,-50.630885,-47.241750,-52.390919,-47.591252,-45.309229,-46.717496,-44.098494,-52.912908
6,34.685226,-5.335541,0.061349,5.500696,4.477012,6.507306,-14.929378,-1.676967,5.199579,-4.225476,...,13.984731,7.177011,32.642448,-3.959449,36.698446,-2.701592,21.275775,2.700939,20.907910,8.665082
7,2.485248,11.021747,1.542594,15.100955,-0.551958,8.309821,28.184881,71.094586,-1.911375,77.805742,...,-33.867612,-116.085425,1.324527,9.581395,3.552758,11.953852,-1.835808,12.648520,2.734961,16.160601
8,16.499639,-6.491380,11.157842,9.059501,-7.814514,7.848489,-9.217319,22.525462,1.736652,6.531698,...,-25.153666,29.877914,11.714411,-12.103554,17.926162,-14.525578,7.756132,-14.972414,23.698983,-22.019019
9,8.920644,0.665588,-7.491678,-13.153432,20.687703,-20.486810,-16.260869,-51.477119,-1.843767,-41.750000,...,-152.891624,40.551186,10.394531,2.094083,8.625598,8.962868,7.380012,-0.445166,-5.520497,17.848897


# Step 4: Assign dance move labels 

# Step 5: Convert pose DataFrame to movement Vectors